In [ ]:
## GEOG458
## Xinbo Lu, Wenjin Yu, Jody Tran
## 2/15/2019
## Group Lab 3
import sys
sys.path.append('D:\\ArcMap\\Arcmap\\Desktop10.5\\bin')
sys.path.append('D:\\ArcMap\\Arcmap\\Desktop10.5\\arcpy')
sys.path.append('D:\\ArcMap\\Arcmap\\Desktop10.5\\\\ArcToolbox\\Scripts')

In [1]:
import geopandas as gpd
import matplotlib as mpl
%matplotlib inline

In [19]:
WA_bg = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/saep_bg10_Copy/saep_bg10.shp")
FIPS_to_CountyNames_gdf = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/WashingtonFIPS_copy.dbf")
WA_bg.head(5)

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,OHU2014,OHU2015,OHU2016,OHU2017,OHU2018,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry
0,53,001,950100,1,530019501001,Block Group 1,G5030,S,-118.398815,47.150809,...,277.494,279.680,287.530,287.522,287.758,33.751,0.143,17.758,0.066,"POLYGON ((2077217.074095237 640954.0110817049,..."
1,53,001,950100,2,530019501002,Block Group 2,G5030,S,-118.351143,47.135101,...,524.831,521.228,517.427,516.077,524.645,3.398,0.007,-0.355,0.000,"POLYGON ((2165913.051240579 657352.8005365322,..."
2,53,001,950100,3,530019501003,Block Group 3,G5030,S,-118.382148,47.138222,...,246.344,243.965,242.033,241.365,245.491,-26.312,-0.097,-0.509,-0.002,"POLYGON ((2166254.454566129 657810.7137267586,..."
3,53,001,950200,1,530019502001,Block Group 1,G5030,S,-118.219484,46.874027,...,208.923,203.761,205.502,204.547,202.854,-21.255,-0.093,-5.146,-0.025,"POLYGON ((2150159.78406948 554295.8999707697, ..."
4,53,001,950200,2,530019502002,Block Group 2,G5030,S,-118.522323,46.968569,...,285.857,286.251,278.398,278.400,267.983,-1.330,-0.005,-22.017,-0.076,"POLYGON ((2082999.834788324 575018.8611698836,..."


In [21]:
print "Initial table: "
print FIPS_to_CountyNames_gdf.head(5)

Initial table: 
  CountyName FIPSCounty geometry
0      Adams        001     None
1     Asotin        003     None
2     Benton        005     None
3     Chelan        007     None
4    Clallam        009     None


In [22]:
FIPS_to_CountyNames_gdf.rename(columns={'FIPSCounty':'COUNTYFP10'},inplace=True)
FIPS_to_CountyNames_gdf.drop('geometry', axis=1, inplace=True)
print
print "After preparation for merge (join): "
print FIPS_to_CountyNames_gdf.head()


After preparation for merge (join): 
  CountyName COUNTYFP10
0      Adams        001
1     Asotin        003
2     Benton        005
3     Chelan        007
4    Clallam        009


In [24]:
JOINING_KEY = 'COUNTYFP10'
WA_bg_with_CountyName = WA_bg.merge(FIPS_to_CountyNames_gdf, on=JOINING_KEY)
WA_bg_with_CountyName.head()
type(WA_bg_with_CountyName)

geopandas.geodataframe.GeoDataFrame

In [20]:
WA_bg_with_CountyName_wgs84 = WA_bg_with_CountyName.to_crs(epsg=4326)
import os  
for county in FIPS_to_CountyNames_gdf['CountyName']:
    filename_to_output = county+".geojson"
    print "Outputting "+filename_to_output
    try: 
        os.remove(filename_to_output)
    except OSError:
        pass
    (WA_bg_with_CountyName_wgs84[WA_bg_with_CountyName['CountyName'] == county]
     .to_file(filename_to_output,driver="GeoJSON"))

Outputting Adams.geojson
Outputting Asotin.geojson
Outputting Benton.geojson
Outputting Chelan.geojson
Outputting Clallam.geojson
Outputting Clark.geojson
Outputting Columbia.geojson
Outputting Cowlitz.geojson
Outputting Douglas.geojson
Outputting Ferry.geojson
Outputting Franklin.geojson
Outputting Garfield.geojson
Outputting Grant.geojson
Outputting Grays Harbor.geojson
Outputting Island.geojson
Outputting Jefferson.geojson
Outputting King.geojson
Outputting Kitsap.geojson
Outputting Kittitas.geojson
Outputting Klickitat.geojson
Outputting Lewis.geojson
Outputting Lincoln.geojson
Outputting Mason.geojson
Outputting Okanogan.geojson
Outputting Pacific.geojson
Outputting Pend Oreille.geojson
Outputting Pierce.geojson
Outputting San Juan.geojson
Outputting Skagit.geojson
Outputting Skamania.geojson
Outputting Snohomish.geojson
Outputting Spokane.geojson
Outputting Stevens.geojson
Outputting Thurston.geojson
Outputting Wahkiakum.geojson
Outputting Walla Walla.geojson
Outputting Whatcom.g

In [22]:
## Print out a ranked list, in descending order, of the ten largest total populations 
## in 2017 of counties in Washington, according to this dataset.

pop2017_series = WA_bg_with_CountyName.groupby(['CountyName']).sum()['POP2017'].rank(ascending=False).head(10)
top10_POP2017_county = pop2017_series.index.tolist()

In [68]:
## Part 3
WA_shp = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/saep_bg10_Copy/saep_bg10.shp")
WA_shp.head(5)
WA_shp['Pop_Per_sqML'] = WA_bg_with_CountyName.POP2017/WA_bg.ALANDMI + 10**-8

WA_shp['Urban'] =""

In [69]:
for values in WA_shp.iterrows():
    if(WA_shp.Pop_Per_sqML[values] >= 500):
        WA_shp.Urban[values] = "TRUE"
    else:
        WA_shp.Urban[values] = "FALSE"

ValueError: Can only tuple-index with a MultiIndex

In [33]:
##WA_shp['Pop_Per_sqML'] = WA_bg_with_CountyName.POP2017/WA_bg.ALANDMI + 10**-8
UrbanArea = WA_shp[(WA_shp['Pop_Per_sqML'] >= 500)]
UrbanPopPercent = round(sum(UrbanArea.POP2017 / sum(WA_shp.POP2017)) * 100, 2)
print "The percentage of the population of the state is urbanized in 2017 is", UrbanPopPercent, "%"

The percentage of the population of the state is urbanized in 2017 is 80.16 %
